<a href="https://colab.research.google.com/github/rsemihkoca/YouDo-ds-bc/blob/main/You_Do_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
import numpy as np
import pandas as pd
df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t',names=['user_id', 'item_id', 'rating', 'timestamp'])
r = df.pivot(index='user_id', columns='item_id', values='rating').values



/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


# Part 0 Train-Test Split

In [126]:
irow, jcol = np.where(~np.isnan(r))
n=int(len(irow)*0.2)

idx = np.random.choice(np.arange(len(irow)), n, replace=False)

In [127]:
test_irow = irow[idx]
test_jcol = jcol[idx]
jcol[idx].shape

(20000,)

In [128]:
r_original=r.copy()
r_copy = r.copy()
l=0
for i in range(n):
  r_copy[test_irow[i]][test_jcol[i]] = np.nan
  l+=1

In [129]:
np.isnan(r_copy).sum()-np.isnan(r_original).sum()

20000

In [130]:
users,items=r_copy.shape

In [131]:
def train_loss(r,b_users,b_items,irow, jcol):
  Train_Loss=0
  for u, j in zip(irow, jcol):
    Train_Loss+=np.nansum((r[u][j] - (b_users[u] + b_items[j]))**2)*0.5
  return Train_Loss

def test_loss(r,b_users,b_items,test_irow, test_jcol):
  Test_Loss=0
  for u, j in zip(test_irow, test_jcol):
    Test_Loss+=np.nansum((r[u][j] -  (b_users[u] + b_items[j]))**2)*0.5
  return Test_Loss

# Part 1 Optimization 

In [132]:
b_users=np.random.rand(users,1)
grad_b_users=np.random.rand(users,1)

b_items=np.random.rand(items,1)
grad_b_items=np.random.rand(items,1)

alpha=(3)*(10**-2)
max_iter=100
r_pred=np.full((r_copy.shape),np.nan)

for iter in range(max_iter):
  for i in range(users):
    for j in range(items):
      if np.isnan(r_copy[i][j]): # zaten bilmediğim değeri tahmin edemem NANları nasıl tahmin ederim
            continue
      else:
        try:

          r_pred[i][j]= b_users[i] + b_items[j]

          grad_b_users[i]=-(r_copy[i][j]-r_pred[i][j])
          grad_b_items[j]=-(r_copy[i][j]-r_pred[i][j])

          b_users_prev=b_users.copy()

          b_users[i] = b_users[i]- alpha * grad_b_users[i]
          b_items[j] = b_items[j]- alpha * grad_b_items[j]


        except ValueError:
          print("ValueError encountered")
  Train_Loss= train_loss(r_copy,b_users,b_items,irow, jcol)
  Test_Loss= test_loss(r_original,b_users,b_items,test_irow, test_jcol)

  if(iter%1 == 0):
    print(f"iteration: ({iter}) , gradient: {np.linalg.norm(grad_b_users)+np.linalg.norm(grad_b_items):.3f}, Train_Loss: {Train_Loss:.3f}, Test_Loss: {Test_Loss:.3f}")
            
  # if np.linalg.norm(b_users-b_users_prev) < 0.000001:
  #   print(f"I do early stoping at iteration {iter}")
  #   break


iteration: (0) , gradient: 96.649, Train_Loss: 82508.109, Test_Loss: 21168.831
iteration: (1) , gradient: 81.200, Train_Loss: 38516.981, Test_Loss: 10256.688
iteration: (2) , gradient: 75.962, Train_Loss: 36030.552, Test_Loss: 9643.081
iteration: (3) , gradient: 73.306, Train_Loss: 35253.462, Test_Loss: 9451.695
iteration: (4) , gradient: 71.648, Train_Loss: 34901.887, Test_Loss: 9366.439
iteration: (5) , gradient: 70.476, Train_Loss: 34708.773, Test_Loss: 9321.199
iteration: (6) , gradient: 69.581, Train_Loss: 34588.120, Test_Loss: 9294.342
iteration: (7) , gradient: 68.865, Train_Loss: 34505.539, Test_Loss: 9277.127
iteration: (8) , gradient: 68.272, Train_Loss: 34445.172, Test_Loss: 9265.491
iteration: (9) , gradient: 67.771, Train_Loss: 34398.883, Test_Loss: 9257.329
iteration: (10) , gradient: 67.339, Train_Loss: 34362.121, Test_Loss: 9251.454
iteration: (11) , gradient: 66.963, Train_Loss: 34332.149, Test_Loss: 9247.151
iteration: (12) , gradient: 66.632, Train_Loss: 34307.219, T

In [133]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

a=pd.DataFrame(r_pred[:5,:10]).fillna('-', inplace=False)
b=pd.DataFrame(r_copy[:5,:10]).fillna('-', inplace=False)
print("                  r_pred vs r_copy\n")
pd.concat([a,b],axis=0)

                  r_pred vs r_copy



,0,1,2,3,4,5,6,7,8,9
0,-,3.15,2.77,3.40,3.09,3.78,3.75,3.85,3.70,3.65
1,3.88,-,-,-,-,-,-,-,-,3.89
2,-,-,-,-,-,-,-,-,-,-
3,-,-,-,-,-,-,-,-,-,-
4,3.30,-,-,-,-,-,-,-,-,-
0,-,3.00,4.00,3.00,3.00,5.00,4.00,1.00,5.00,3.00
1,4.00,-,-,-,-,-,-,-,-,2.00
2,-,-,-,-,-,-,-,-,-,-
3,-,-,-,-,-,-,-,-,-,-
4,4.00,-,-,-,-,-,-,-,-,-


# Part 2 Regularized Model

In [134]:
def train_loss_regularized(r,b_users,b_items,irow, jcol,lamb):
  Train_Loss=0
  for u, j in zip(irow, jcol):
    Train_Loss+=np.nansum((r[u][j] - (b_users[u] + b_items[j]))**2)*0.5 + (b_users[u]**2 + b_items[j]**2)*lamb*0.5
  return float(Train_Loss[0])

def test_loss_regularized(r,b_users,b_items,test_irow, test_jcol,lamb):
  Test_Loss=0
  for u, j in zip(test_irow, test_jcol):
    Test_Loss+=np.nansum((r[u][j] -  (b_users[u] + b_items[j]))**2)*0.5 + (b_users[u]**2 + b_items[j]**2)*lamb*0.5
  return float(Test_Loss[0])



In [135]:
import numpy as np




def Regularized_SGD(lamb):

  b_users=np.random.rand(users,1)
  grad_b_users=np.random.rand(users,1)

  b_items=np.random.rand(items,1)
  grad_b_items=np.random.rand(items,1)

  r_pred=np.full((r_copy.shape),np.nan)

  alpha=(3)*(10**-2)
  Test_Loss_list=[]
  Info=False
  max_iter=3

  for iter in range(max_iter):
    for i in range(users):
      for j in range(items):
        if np.isnan(r_copy[i][j]): # zaten bilmediğim değeri tahmin edemem NANları nasıl tahmin ederim
              continue
        else:
          try:

            r_pred[i][j]= b_users[i] + b_items[j]

            grad_b_users[i]=-(r_copy[i][j]-r_pred[i][j]) + lamb * b_users[i]
            grad_b_items[j]=-(r_copy[i][j]-r_pred[i][j]) + lamb * b_items[j]

            b_users_prev=b_users.copy()

            b_users[i] = b_users[i]- alpha * grad_b_users[i]
            b_items[j] = b_items[j]- alpha * grad_b_items[j]


          except ValueError:
            print("ValueError encountered")
    Train_Loss= train_loss_regularized(r_copy,b_users,b_items,irow, jcol,lamb)
    Test_Loss= test_loss_regularized(r_original,b_users,b_items,test_irow, test_jcol,lamb)
    Test_Loss_list.append(Test_Loss)
    if Info:
      if(iter%1 == 0):
        print(f"iteration: ({iter}) , gradient: {(np.linalg.norm(grad_b_users)+np.linalg.norm(grad_b_items)):.3f}, Reg_Train_Loss: {Train_Loss:.3f}, Reg_Test_Loss: {Test_Loss:.3f}")
        
  return -min(Test_Loss_list)

max_iter=3
lamb=0.001

Regularized_SGD(lamb)


    # return r_pred,b_users, b_items

-9737.001887101575

# Part 2 HyperParam Opt

In [136]:
!pip install bayesian-optimization

In [137]:

#Regularized_SGD(max_iter,lamb,Info=False)

from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'lamb': (0.00005,0.1)}

optimizer = BayesianOptimization(
    f=Regularized_SGD,
    pbounds=pbounds,
    random_state=1,
)

In [138]:
optimizer.maximize(
    init_points=0.0001,
    n_iter=10,
)

|   iter    |  target   |   lamb    |
-------------------------------------
|  1        | -1.228e+0 |  0.04173  |
|  2        | -1.581e+0 |  0.1      |
|  3        | -1.228e+0 |  0.04174  |
|  4        | -1.196e+0 |  0.03687  |
|  5        | -1.144e+0 |  0.02818  |
|  6        | -1.108e+0 |  0.02262  |
|  7        | -9.635e+0 |  5e-05    |
|  8        | -9.635e+0 |  5e-05    |
|  9        | -9.635e+0 |  5e-05    |
|  10       | -9.635e+0 |  5e-05    |
|  11       | -9.635e+0 |  5e-05    |


In [139]:
print(f"Best lambda value: {optimizer.max['params']['lamb']:.5f}\nRegular_Test_Loss: {-optimizer.max['target']:.5f}")


Best lambda value: 0.00005
Regular_Test_Loss: 9635.30415
